In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import cv2
import os
import sys
from PIL import Image
from skimage import morphology
from skimage.morphology import skeletonize
from keras_unet.utils_regine import plot_imgs_regine

## Load data

In [ ]:
#masks = glob.glob("../input/graph_images/train/label/*.png")
masks = np.load('S:/06_Studienarbeit/03_CNN/generate_data/data/train/label/adjcouput_matrix.npy',allow_pickle='TRUE').item()

orgs = glob.glob("S:/06_Studienarbeit/03_CNN/generate_data/data/train/image/*.png")
#orgs = orgs[:10]

In [ ]:
imgs_list = []
masks_list = []
for image, mask in zip(orgs, masks):

    I = cv2.imread(image)
    #normalize positions
    key = image
    print(key[-14:-9])
    graph_label = masks[key[-14:-9]]
    graph_label_norm = graph_label.copy()
    positions = graph_label[:, :2, 0]
    pos_norm = np.zeros(positions.shape)
    for i in range(len(positions)):
        pos_norm[i][0] = np.round((positions[i][0]/I.shape[1])*512, 0)
        pos_norm[i][1] = np.round((positions[i][1] /I.shape[0])*512, 0)
    graph_label_norm[:, :2, 0] = pos_norm

    imgs_list.append(np.array(Image.open(image).convert('L').resize((512,512))))
    print(image)

    #im = Image.open(mask).resize((512,512))


    #width, height = im.size   # Get dimensions

    #left = (width - 388)/2
    #top = (height - 388)/2
    #right = (width + 388)/2
    #bottom = (height + 388)/2

    #im_cropped = im.crop((left, top, right, bottom))
    masks_list.append(np.array(graph_label_norm))

imgs_np = np.asarray(imgs_list)
masks_np = np.asarray(masks_list)

In [ ]:
print(imgs_np.shape, masks_np.shape)

## Plot images + masks + overlay (mask over original)

In [ ]:
#plot_imgs(org_imgs=imgs_np, mask_imgs=masks_np, nm_img_to_plot=10, figsize=6)
import time
time_start = time.time()
nm_img_to_plot = 2
fig = plot_imgs_regine(imgs_np, masks_np, 2)
plt.show()
time_end = time.time()
plotting_time = time_end-time_start

## Get data into correct shape, dtype and range (0.0-1.0)

In [ ]:
#print(imgs_np.max(), masks_np.max())

In [ ]:
x = np.asarray(imgs_np, dtype=np.float32)/255
y = np.asarray(masks_np, dtype=np.float32)/255

In [ ]:
print(x.max(), y.max())

In [ ]:
print(x.shape, y.shape)

In [ ]:
y = y.reshape(y.shape[0], y.shape[1], y.shape[2], 1)
print(x.shape, y.shape)

In [ ]:
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
print(x.shape, y.shape)

##  Train/val split

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.5, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

## Prepare train generator with data augmentation

In [ ]:
#from keras_unet.utils import get_augmented
#
# train_gen = get_augmented(
#     x_train, y_train, batch_size=2,
#     data_gen_args = dict(
#         rotation_range=15.,
#         width_shift_range=0.05,
#         height_shift_range=0.05,
#         shear_range=50,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         vertical_flip=True,
#         fill_mode='constant'
#     ))


In [ ]:
train_gen = zip(x_train, y_train)

sample_batch = next(train_gen)
xx, yy = sample_batch
print(xx.shape, yy.shape)
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=xx, mask_imgs=yy, nm_img_to_plot=2, figsize=6)

## Initialize network

In [ ]:
# from keras_unet.models import custom_unet
#
input_shape = x_train[0].shape
#
# model = custom_unet(
#     input_shape,
#     upsample_mode="deconv",  # 'deconv' or 'simple'
#     use_batch_norm=False,
#     num_classes=1,
#     filters=64,
#     dropout=0.2,
#     output_activation='relu'
# )

In [ ]:
#model.summary()

In [ ]:
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

#from models_graph import *
from models_graph.custom_graph_head import custom_graph_head, custom_adj_unet

print(input_shape)
model = custom_adj_unet(input_size = input_shape, pretrained_weights =None)


In [ ]:
model.summary()

In [ ]:
import os
os.environ["PATH"] += os.pathsep + "C:\\Program Files\\Graphviz\\bin\\"

In [ ]:
import sys
print(sys.path)

sys.path.append("C:\\Program Files\\Graphviz\\bin\\")

print(sys.path)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

## Compile + train

In [ ]:
from keras.callbacks import ModelCheckpoint


model_filename = 'segm_model_v0.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [ ]:
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
    #optimizer=Adam(), 
    optimizer=SGD(lr=0.01, momentum=0.99),
    loss='binary_crossentropy',
    #loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)


In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=10,
    
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint]
)

## Plot training history

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

## Plot original + ground truth + pred + overlay (pred on top of original)

In [ ]:
model.load_weights(model_filename)
y_pred = model.predict(x_val)

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=x_val, mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=9)